# Pacote

In [1]:
import pandas as pd
import chardet
import glob
from unidecode import unidecode
import os
import Levenshtein as lev
import csv 

pd.set_option('display.max_columns', None)

In [2]:
import sys

print(os.environ.get("SPARK_HOME"))
print(os.environ.get("HADOOP_HOME"))
print(os.environ.get("JAVA_HOME"))

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

C:\Users\pedro\spark-3.5.0-bin-hadoop3
C:\Users\pedro\hadoop3.0
C:\Program Files\Java\jdk1.8.0_202


In [3]:
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.types import *
from pyspark.sql.functions import lower, upper,row_number,isnan, when, count, col, coalesce, broadcast, regexp_replace, regexp_extract, lit, countDistinct
from pyspark.sql import functions as F, Window, Row
from pyspark.sql.functions import *
#from functools import reduce

#Pyspark
import py4j
from pyspark import SparkContext,SQLContext,SparkConf,StorageLevel

## Pacotes para configurar sessão no spark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
                            
## Pacote para localizar o path spark 
import findspark

from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
import unicodedata

## Spark session

In [4]:
# Usa todos os núcleos disponíveis na máquina local.
# Define o nome da aplicação.
# Número de núcleos alocados para o driver Spark.
# Quantidade de memória alocada para o driver Spark.
# Nível de paralelismo padrão para todas as transformações em RDDs.
# Número de partições para usar quando fazer operações de shuffle.
# Número de instâncias do executor para iniciar.
# Número de núcleos para usar por executor.
# Quantidade de memória alocada para cada executor.
# Fração da heap do executor para armazenamento e execução.
# Proporção da memória de execução acima da qual o armazenamento será despejado para o disco.
# Habilita o uso de memória fora do heap.
# Tamanho da memória fora do heap alocada para o Spark.
# Tamanho máximo dos resultados do driver.
# Memória adicional alocada por executor.
# Habilita a avaliação antecipada e a visualização dos DataFrames no Spark SQL REPL.
# Número máximo de linhas para mostrar quando a avaliação antecipada está habilitada.
# Tamanho máximo do buffer para serialização Kryo.
# Tamanho máximo das tabelas na realização do broadcast join 
# Usa KryoSerializer para serialização, oferecendo melhor desempenho.
# Classe de registrator Kryo para registrar classes personalizadas com Kryo.
# Comprime os dados shuffle para economizar espaço em disco.
# Define o nível de armazenamento para RDDs persistidos, usando tanto a memória quanto o disco.
# Comprime RDDs armazenados em memória.

spark = (SparkSession.builder 
    .master("local[*]") 
    .appName("Spark Optimization")   
    .config("spark.driver.cores", "2")   
    .config("spark.driver.memory", "8g")   
    .config("spark.default.parallelism", "24")   
    .config("spark.sql.shuffle.partitions", "24")   
    .config("spark.executor.instances", "3")   
    .config("spark.executor.cores", "2")   
    .config("spark.executor.memory", "10g")   
    .config("spark.memory.fraction", "0.6")  
    .config("spark.memory.storageFraction", "0.5")   
    .config("spark.memory.offHeap.enabled", "true")   
    .config("spark.memory.offHeap.size", "4g")   
    .config("spark.driver.maxResultSize", "4g")   
    .config("spark.executor.memoryOverhead", "2g")   
    .config("spark.sql.repl.eagerEval.enabled", True)   
    .config("spark.sql.repl.eagerEval.maxNumRows", 10)  
#    .config("spark.kryoserializer.buffer.max", "512m")  
    .config("spark.sql.autoBroadcastJoinThreshold", "400m")   
#    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")   
#    .config("spark.kryo.registrator", "MyKryoRegistrator")   
    .config("spark.shuffle.compress", "true")   
    .config("spark.storage.level", "MEMORY_AND_DISK")   
    .config("spark.rdd.compress", "true")   
    .getOrCreate())

In [5]:
spark

# Carregando dados

## Df meteorologia

In [75]:
df = (spark.read.parquet("DADOS_METEROLOGIA/DADOS_METEOROLOGICOS_TRATADOS/dados_meteorologicos_2023.parquet").repartition(12))
df = df.withColumn("hora_utc", col("hora_utc").cast("int"))
df = df.withColumnRenamed("estacao", "cidade")

In [76]:
# Renomeando as colunas para aeroportos de origem
df_origem = df.select(
    *[col(c).alias(c + '_origem') for c in df.columns])

# Renomeando as colunas para aeroportos de destino
df_destino = df.select(
    *[col(c).alias(c + '_destino') for c in df.columns])

df_origem = df_origem.withColumnRenamed("data_origem", "data_partida") \
                 .withColumnRenamed("hora_utc_origem", "hora_partida")

df_destino = df_destino.withColumnRenamed("data_destino", "data_chegada")\
                 .withColumnRenamed("hora_utc_destino", "hora_chegada")

In [77]:
df_destino.printSchema()

root
 |-- data_chegada: string (nullable = true)
 |-- hora_chegada: integer (nullable = true)
 |-- cidade_destino: string (nullable = true)
 |-- uf_destino: string (nullable = true)
 |-- regiao_destino: string (nullable = true)
 |-- precipitacao_total_horario_mm__destino: double (nullable = true)
 |-- pressao_atmosferica_ao_nivel_da_estacao_horaria_mb__destino: double (nullable = true)
 |-- pressao_atmosferica_max_na_hora_ant_aut_mb__destino: double (nullable = true)
 |-- pressao_atmosferica_min_na_hora_ant_aut_mb__destino: double (nullable = true)
 |-- temperatura_do_ar_bulbo_seco_horaria_degc__destino: double (nullable = true)
 |-- temperatura_do_ponto_de_orvalho_degc__destino: double (nullable = true)
 |-- temperatura_maxima_na_hora_ant_aut_degc__destino: double (nullable = true)
 |-- temperatura_minima_na_hora_ant_aut_degc__destino: double (nullable = true)
 |-- temperatura_orvalho_max_na_hora_ant_aut_degc__destino: double (nullable = true)
 |-- temperatura_orvalho_min_na_hora_ant_

## Df voos

In [78]:
df_voos=spark.read.option("header", "true").csv("dados_tratados/historico_voo_tratados_2023.csv").repartition(6)

df_voos = df_voos.withColumn('partida_prevista_data', to_date(col('partida_prevista')))
df_voos = df_voos.withColumn('chegada_prevista_data', to_date(col('chegada_prevista')))

In [79]:
df_voos = df_voos.withColumn("hora_chegada", col("hora_chegada").cast("int"))
df_voos = df_voos.withColumn("hora_partida", col("hora_partida").cast("int"))

In [80]:
df_voos

nome_empresas,numero_voo,codigo_di,codigo_tipo_linha,partida_prevista,partida_real,chegada_prevista,chegada_real,situacao_voo,descricao_origem,pais_origem,continente_origem,descricao_destino,pais_destino,continente_destino,status_do_voo,delta_tempo_partida_delta,delta_tempo_chegada_delta,delta_tempo_partida_int,delta_tempo_chegada_int,cidade_origem,uf_origem_x,cidade_destino,uf_destino_x,mes_partida,dia_semana_partida,dia_mes_partida,hora_partida,mes_chegada,dia_semana_chegada,dia_mes_chegada,hora_chegada,altitude_origem,latgeopoint_origem,longeopoint_origem,latitude_aero_origem,longitude_aero_origem,altitude_destino,latgeopoint_destino,longeopoint_destino,latitude_aero_destino,longitude_aero_destino,rota,partida_prevista_data,chegada_prevista_data
TAM,8033,0,I,2022-01-18 12:35:00,NULL,2022-01-18 15:10:00,NULL,NULL,JORGE NEWBERY AIR...,ARGENTINA,AMERICA DO SUL,GUARULHOS - GOVER...,BRASIL,AMERICA DO SUL,Pontual,0,0,0,0,BUENOS AIRES/AERO...,NULL,GUARULHOS,SP,1.0,1.0,18.0,12,1.0,1.0,18.0,15,NULL,NULL,NULL,NULL,NULL,750.0,-23.435.556,-46.473.056,-23.43555555555556,-46.473055555555554,JORGE NEWBERY AIR...,2022-01-18,2022-01-18
TAM,3309,0,N,2022-12-04 19:20:00,2022-12-04 19:20:00,2022-12-04 22:50:00,2022-12-04 22:48:00,REALIZADO,PINTO MARTINS,BRASIL,AMERICA DO SUL,GUARULHOS - GOVER...,BRASIL,AMERICA DO SUL,Pontual,0 days 00:00:00,0 days 00:02:00,0,2,FORTALEZA,CE,GUARULHOS,SP,12.0,6.0,4.0,19,12.0,6.0,4.0,22,25.0,-37.758.333,-38.532.222,-3.775833333333333,-38.532222222222224,750.0,-23.435.556,-46.473.056,-23.43555555555556,-46.473055555555554,PINTO MARTINS -> ...,2022-12-04,2022-12-04
TAM,3321,0,N,2022-12-21 16:05:00,2022-12-21 16:02:00,2022-12-21 17:10:00,2022-12-21 17:17:00,REALIZADO,AEROPORTO INTERNA...,BRASIL,AMERICA DO SUL,GUARULHOS - GOVER...,BRASIL,AMERICA DO SUL,Pontual,0 days 00:03:00,0 days 00:07:00,3,7,RIO DE JANEIRO,RJ,GUARULHOS,SP,12.0,2.0,21.0,16,12.0,2.0,21.0,17,9.0,-22.81,-43.250.556,-22.81,-43.25055555555556,750.0,-23.435.556,-46.473.056,-23.43555555555556,-46.473055555555554,AEROPORTO INTERNA...,2022-12-21,2022-12-21
AZUL,2768,0,N,2022-09-21 13:25:00,2022-09-21 13:19:00,2022-09-21 16:40:00,2022-09-21 16:36:00,REALIZADO,GUARARAPES - GILB...,BRASIL,AMERICA DO SUL,GUARULHOS - GOVER...,BRASIL,AMERICA DO SUL,Pontual,0 days 00:06:00,0 days 00:04:00,6,4,RECIFE,PE,GUARULHOS,SP,9.0,2.0,21.0,13,9.0,2.0,21.0,16,10.0,-81.263.889,-34.922.778,-8.12638888888889,-34.922777777777775,750.0,-23.435.556,-46.473.056,-23.43555555555556,-46.473055555555554,GUARARAPES - GILB...,2022-09-21,2022-09-21
GOL,1251,0,N,2022-07-18 18:15:00,2022-07-18 18:33:00,2022-07-18 19:35:00,2022-07-18 20:03:00,REALIZADO,HERCILIO LUZ,BRASIL,AMERICA DO SUL,GUARULHOS - GOVER...,BRASIL,AMERICA DO SUL,Atrasado,0 days 00:18:00,0 days 00:28:00,18,28,FLORIANOPOLIS,SC,GUARULHOS,SP,7.0,0.0,18.0,18,7.0,0.0,18.0,19,5.0,-27.670.278,-485.525,-27.67027777777778,-48.5525,750.0,-23.435.556,-46.473.056,-23.43555555555556,-46.473055555555554,HERCILIO LUZ -> G...,2022-07-18,2022-07-18
TAM,8045,0,I,2022-07-06 11:15:00,2022-07-06 11:21:00,2022-07-06 13:40:00,2022-07-06 14:33:00,REALIZADO,JORGE NEWBERY AIR...,ARGENTINA,AMERICA DO SUL,GUARULHOS - GOVER...,BRASIL,AMERICA DO SUL,Pontual,0 days 00:06:00,0 days 00:53:00,6,53,BUENOS AIRES/AERO...,NULL,GUARULHOS,SP,7.0,2.0,6.0,11,7.0,2.0,6.0,13,NULL,NULL,NULL,NULL,NULL,750.0,-23.435.556,-46.473.056,-23.43555555555556,-46.473055555555554,JORGE NEWBERY AIR...,2022-07-06,2022-07-06
TAM,3423,0,N,2022-06-05 13:15:00,2022-06-05 13:10:00,2022-06-05 14:55:00,2022-06-05 14:57:00,REALIZADO,PRESIDENTE JUSCEL...,BRASIL,AMERICA DO SUL,GUARULHOS - GOVER...,BRASIL,AMERICA DO SUL,Pontual,0 days 00:05:00,0 days 00:02:00,5,2,BRASILIA,DF,GUARULHOS,SP,6.0,6.0,5.0,13,6.0,6.0,5.0,14,1066.0,-15.871.111,-47.918.611,-15.871111111111112,-47.91861111111111,750.0,-23.435.556,-46.473.056,-23.43555555555556,-46.473055555555554,PRESIDENTE JUSCEL...,2022-06-05,2022-06-05
TAM,8045,0,I,2022-07-26 11:45:00,2022-07-26 11:41:00,2022-07-26 14:20:00,2022-07-26 14:06:00,REALIZAD

In [81]:
df_voos.printSchema()

root
 |-- nome_empresas: string (nullable = true)
 |-- numero_voo: string (nullable = true)
 |-- codigo_di: string (nullable = true)
 |-- codigo_tipo_linha: string (nullable = true)
 |-- partida_prevista: string (nullable = true)
 |-- partida_real: string (nullable = true)
 |-- chegada_prevista: string (nullable = true)
 |-- chegada_real: string (nullable = true)
 |-- situacao_voo: string (nullable = true)
 |-- descricao_origem: string (nullable = true)
 |-- pais_origem: string (nullable = true)
 |-- continente_origem: string (nullable = true)
 |-- descricao_destino: string (nullable = true)
 |-- pais_destino: string (nullable = true)
 |-- continente_destino: string (nullable = true)
 |-- status_do_voo: string (nullable = true)
 |-- delta_tempo_partida_delta: string (nullable = true)
 |-- delta_tempo_chegada_delta: string (nullable = true)
 |-- delta_tempo_partida_int: string (nullable = true)
 |-- delta_tempo_chegada_int: string (nullable = true)
 |-- cidade_origem: string (nullable =

In [82]:
df_voos = df_voos.withColumnRenamed("partida_prevista_data", "data_partida") \
                 .withColumnRenamed("hora_partida", "hora_partida") \
                 .withColumnRenamed("cidade_origem", "cidade_origem") \
                 .withColumnRenamed("uf_origem_x", "uf_origem")

df_voos = df_voos.withColumnRenamed("chegada_prevista_data", "data_chegada") \
                 .withColumnRenamed("uf_destino_x", "uf_destino")

# Convertendo as colunas data_partida e data_chegada para string
df_voos = df_voos.withColumn("data_partida", col("data_partida").cast("string")) \
       .withColumn("data_chegada", col("data_chegada").cast("string"))

## Join entre os datasets

In [83]:
# Colunas para realizar o join
join_cols = ['data_chegada', 'hora_chegada', 'cidade_destino', 'uf_destino']

# Realizando o join
df_joined = df_voos.join(df_destino, on=join_cols, how='left').drop(df_destino.data_chegada, df_destino.hora_chegada, df_destino.cidade_destino, df_destino.uf_destino, df_destino.altitude_destino)

# Colunas para realizar o join
join_cols = ['data_partida','hora_partida','cidade_origem','uf_origem']

# Realizando o join
df_joined_final = df_joined.join(df_origem, on=join_cols, how='left').drop(df_origem.data_partida, df_origem.hora_partida, df_origem.cidade_origem, df_origem.uf_origem, df_origem.altitude_origem)

df_joined_final = df_joined_final.dropDuplicates()

In [84]:
df_joined_final = df_joined_final.withColumn("ano", lit(2023))

In [85]:
df_joined_final

data_partida,hora_partida,cidade_origem,uf_origem,data_chegada,hora_chegada,cidade_destino,uf_destino,nome_empresas,numero_voo,codigo_di,codigo_tipo_linha,partida_prevista,partida_real,chegada_prevista,chegada_real,situacao_voo,descricao_origem,pais_origem,continente_origem,descricao_destino,pais_destino,continente_destino,status_do_voo,delta_tempo_partida_delta,delta_tempo_chegada_delta,delta_tempo_partida_int,delta_tempo_chegada_int,mes_partida,dia_semana_partida,dia_mes_partida,mes_chegada,dia_semana_chegada,dia_mes_chegada,altitude_origem,latgeopoint_origem,longeopoint_origem,latitude_aero_origem,longitude_aero_origem,altitude_destino,latgeopoint_destino,longeopoint_destino,latitude_aero_destino,longitude_aero_destino,rota,regiao_destino,precipitacao_total_horario_mm__destino,pressao_atmosferica_ao_nivel_da_estacao_horaria_mb__destino,pressao_atmosferica_max_na_hora_ant_aut_mb__destino,pressao_atmosferica_min_na_hora_ant_aut_mb__destino,temperatura_do_ar_bulbo_seco_horaria_degc__destino,temperatura_do_ponto_de_orvalho_degc__destino,temperatura_maxima_na_hora_ant_aut_degc__destino,temperatura_minima_na_hora_ant_aut_degc__destino,temperatura_orvalho_max_na_hora_ant_aut_degc__destino,temperatura_orvalho_min_na_hora_ant_aut_degc__destino,umidade_rel_max_na_hora_ant_aut_pcnt__destino,umidade_rel_min_na_hora_ant_aut_pcnt__destino,umidade_relativa_do_ar_horaria_pcnt__destino,vento_direcao_horaria_gr_deg_gr__destino,vento_rajada_maxima_ms__destino,vento_velocidade_horaria_ms__destino,regiao_origem,precipitacao_total_horario_mm__origem,pressao_atmosferica_ao_nivel_da_estacao_horaria_mb__origem,pressao_atmosferica_max_na_hora_ant_aut_mb__origem,pressao_atmosferica_min_na_hora_ant_aut_mb__origem,temperatura_do_ar_bulbo_seco_horaria_degc__origem,temperatura_do_ponto_de_orvalho_degc__origem,temperatura_maxima_na_hora_ant_aut_degc__origem,temperatura_minima_na_hora_ant_aut_degc__origem,temperatura_orvalho_max_na_hora_ant_aut_degc__origem,temperatura_orvalho_min_na_hora_ant_aut_degc__origem,umidade_rel_max_na_hora_ant_aut_pcnt__origem,umidade_rel_min_na_hora_ant_aut_pcnt__origem,umidade_relativa_do_ar_horaria_pcnt__origem,vento_direcao_horaria_gr_deg_gr__origem,vento_rajada_maxima_ms__origem,vento_velocidade_horaria_ms__origem,ano
2022-01-18,12,BUENOS AIRES/AERO...,NULL,2022-01-18,15,GUARULHOS,SP,TAM,8033,0,I,2022-01-18 12:35:00,NULL,2022-01-18 15:10:00,NULL,NULL,JORGE NEWBERY AIR...,ARGENTINA,AMERICA DO SUL,GUARULHOS - GOVER...,BRASIL,AMERICA DO SUL,Pontual,0,0,0,0,1.0,1.0,18.0,1.0,1.0,18.0,NULL,NULL,NULL,NULL,NULL,750.0,-23.435.556,-46.473.056,-23.43555555555556,-46.473055555555554,JORGE NEWBERY AIR...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2023
2022-04-13,11,FORTALEZA,CE,2022-04-13,14,GUARULHOS,SP,TAM,3307,0,N,2022-04-13 11:20:00,2022-04-13 11:11:00,2022-04-13 14:50:00,2022-04-13 14:30:00,NULL,PINTO MARTINS,BRASIL,AMERICA DO SUL,GUARULHOS - GOVER...,BRASIL,AMERICA DO SUL,Pontual,0 days 00:09:00,0 days 00:20:00,9,20,4.0,2.0,13.0,4.0,2.0,13.0,25.0,-37.758.333,-38.532.222,-3.775833333333333,-38.532222222222224,750.0,-23.435.556,-46.473.056,-23.43555555555556,-46.473055555555554,PINTO MARTINS -> ...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2023
2022-07-02,10,BRASILIA,DF,2022-07-02,12,GUARULHOS,SP,GOL,1403,0,N,2022-07-02 10:55:00,2022-07-02 10:51:00,2022-07-02 12:35:00,2022-07-02 12:29:00,REALIZADO,PRESIDENTE JUSCEL...,BRASIL,AMERICA DO SUL,GUARULHOS - GOVER...,BRASIL,AMERICA DO SUL,Pontual,0 days 00:04:00,0 days 00:06:00,4,6,7.0,5.0,2.0,7.0,5.0,2.0,1066.0,-15.871.111,-47.918.611,-15.871111111111112,-47.91861111111111,750.0,-23.435.556,-46.473.056,-23.43555555555556,-46.473055555555554,PRESIDENTE JUSCEL...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL

In [86]:
df_joined_final.count()

715402

# Salvando o dataset tratado com dados de meterologia

In [87]:
# Reparticionando o DataFrame para uma única partição
df_joined_final = df_joined_final.repartition(1)

# Salvando o DataFrame em um único arquivo CSV
(df_joined_final.write
     .option("header", "true")
     .mode("overwrite")
     .csv("dados_tratados/historico_voo_meteorologia_left_2023.csv"))